In [4]:
import pandas as pd
import os

In [5]:
PATH = '../data'

In [6]:
def get_shape():
    for file in os.listdir(PATH):
        if file.endswith(".csv"):
            csv_file = os.path.join(PATH, file)
            df = pd.read_csv(csv_file)
            print(f"{csv_file} | Shape: {df.shape}")

### BiLSTM

In [7]:
get_shape()

../data\absence-of-certain-changes-or-events.csv | Shape: (350, 2)
../data\absence-of-certain-changes.csv | Shape: (350, 2)
../data\acceleration.csv | Shape: (350, 2)
../data\access-to-information.csv | Shape: (270, 2)
../data\access.csv | Shape: (310, 2)
../data\accounting-terms.csv | Shape: (370, 2)
../data\additional-agreements.csv | Shape: (400, 2)
../data\additional-documents.csv | Shape: (470, 2)
../data\adjustments.csv | Shape: (360, 2)
../data\affirmative-covenants.csv | Shape: (500, 2)
../data\agreement.csv | Shape: (500, 2)
../data\agreements.csv | Shape: (480, 2)
../data\amendment-and-waiver.csv | Shape: (400, 2)
../data\amendment-waiver.csv | Shape: (380, 2)
../data\amendment.csv | Shape: (370, 2)
../data\amendments-and-waivers.csv | Shape: (330, 2)
../data\amendments-etc.csv | Shape: (340, 2)
../data\amendments-waivers.csv | Shape: (370, 2)
../data\amendments.csv | Shape: (460, 2)
../data\applicable-law.csv | Shape: (420, 2)
../data\application-of-proceeds.csv | Shape: (42

### Attention based Encoder